# Trabájo Practico 2 - Métodos Computacionales

### Camila Cauzzo, Catalina Dolhare

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import time
import random

In [4]:
from derivada_w import fAndG as fAndG_w
from derivada_b import fAndG as fAndG_b

In [5]:
def abrirImagenesEscaladas( carpeta, escala=32 ):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            img = Image.open( os.path.join(carpeta, file) )
            img = img.resize((escala, escala))
            img.convert('1')
            img = np.asarray(img)
            if len(img.shape)==3:
                img = img[:,:,0].reshape((escala**2 )) / 255
            else:
                img = img.reshape((escala**2 )) / 255
            
            imagenes.append( img )

    return imagenes

In [6]:
def balancear_datos():
    imagenes_train_normal = abrirImagenesEscaladas("chest_xray/train/NORMAL")
    imagenes_train_pneumonia = abrirImagenesEscaladas("chest_xray/train/PNEUMONIA")

    cantidad_imagenes = min(len(imagenes_train_normal), len(imagenes_train_pneumonia))
    imagenes_train_normal = imagenes_train_normal[:cantidad_imagenes]
    imagenes_train_pneumonia = imagenes_train_pneumonia[:cantidad_imagenes]

    return [imagenes_train_normal, imagenes_train_pneumonia]

### Ejercicio 1

Como la derivada de la suma es la suma de las derivadas, sumamos las derivadas de cada imagen

In [30]:
def f(i, w, b):
    f1 = np.tanh(np.dot(w,i) +b)
    f2 = f1 + 1
    return f2/2

In [43]:
def L(imagenes, w, b):
    imagenes_normal = imagenes[0]
    imagenes_pneumonia = imagenes[1]

    sumatoria = 0
    for i in range(len(imagenes_normal)):
        t_0 = np.tanh((w).dot(imagenes_normal[i]) + b)
        t_1 = (((1 + t_0) / 2) - 0)
        t_2 = t_1 * t_1
        sumatoria += t_2
    for i in range(len(imagenes_pneumonia)):
        t_0 = np.tanh((w).dot(imagenes_pneumonia[i]) + b)
        t_1 = (((1 + t_0) / 2) - 1)
        t_2 = t_1 * t_1
        sumatoria += t_2
    return sumatoria

In [56]:
def derivada_w(b, imagenes, w):
    imagenes_normal = imagenes[0]
    imagenes_pneumonia = imagenes[1]
    
    sum_dw = np.zeros(imagenes[0][0].shape)
    for i in range(len(imagenes_normal)):
        func, gradient = fAndG_w(b, 0, imagenes_normal[i], w)
        sum_dw += gradient
    for i in range(len(imagenes_pneumonia)):
        func, gradient = fAndG_w(b, 1, imagenes_pneumonia[i], w)
        sum_dw += gradient
    return sum_dw

def derivada_b(b, imagenes, w):
    imagenes_normal = imagenes[0]
    imagenes_pneumonia = imagenes[1]
    print(len(imagenes_normal))
    print(len(imagenes_pneumonia))
    
    sum_db = 0
    for i in range(len(imagenes_normal)):
        print(i)
        func, gradient = fAndG_b(b, 0, imagenes_normal[i], w)
        sum_db += gradient
        print(sum_db)
    for i in range(len(imagenes_pneumonia)):
        print(i)
        func, gradient = fAndG_b(b, 1, imagenes_pneumonia[i], w)
        sum_db += gradient
        print(sum_db)
    print(sum_db)
    return sum_db


### Ejercicio 2

Una vez que tenemos las derivadas parciales, podemos utilizar el metodo de gradiente descendiente

In [10]:
MAX_ITER = 1000 # maximo numero de iteraciones
TOLERANCIA = 0.0000001 # tolerancia para la convergencia

In [50]:
def gradiente_descendiente(imagenes, alpha):
    w_viejo = np.zeros(imagenes[0][0].shape) 
    b_viejo = 0

    iteracion = 0

    while iteracion < 2:
        print("Iteración: ", iteracion, "- Mínimo alcanzado hasta el momento: ", L(imagenes, w_viejo, b_viejo))

        gradiente_dw = derivada_w(b_viejo, imagenes, w_viejo)
        gradiente_db = derivada_b(b_viejo, imagenes, w_viejo)
        print(derivada_b(b_viejo, imagenes, w_viejo))
        print("Gradiente de w: ", gradiente_dw)
        print("Gradiente de b: ", gradiente_db)

        w_nuevo = w_viejo - alpha * gradiente_dw
        b_nuevo = b_viejo - alpha * gradiente_db
        print("Nuevo w: ", w_nuevo)
        print("Nuevo b: ", b_nuevo)

        if(abs(L(imagenes, w_nuevo, b_nuevo)-L(imagenes, w_viejo, b_viejo)) < TOLERANCIA):
            break

        w_viejo = w_nuevo
        b_viejo = b_nuevo
        iteracion += 1

    return w_nuevo, b_nuevo

In [12]:
imagenes = balancear_datos()

In [57]:
w_opt, b_opt = gradiente_descendiente(imagenes, 0.0001)
print("El mínimo local ocurre en w = ", w_opt, "b = ", b_opt, " y L vale: ", L(imagenes, w_opt, b_opt))

Iteración:  0 - Mínimo alcanzado hasta el momento:  674.5
1349
1349
0
0.5
1
1.0
2
1.5
3
2.0
4
2.5
5
3.0
6
3.5
7
4.0
8
4.5
9
5.0
10
5.5
11
6.0
12
6.5
13
7.0
14
7.5
15
8.0
16
8.5
17
9.0
18
9.5
19
10.0
20
10.5
21
11.0
22
11.5
23
12.0
24
12.5
25
13.0
26
13.5
27
14.0
28
14.5
29
15.0
30
15.5
31
16.0
32
16.5
33
17.0
34
17.5
35
18.0
36
18.5
37
19.0
38
19.5
39
20.0
40
20.5
41
21.0
42
21.5
43
22.0
44
22.5
45
23.0
46
23.5
47
24.0
48
24.5
49
25.0
50
25.5
51
26.0
52
26.5
53
27.0
54
27.5
55
28.0
56
28.5
57
29.0
58
29.5
59
30.0
60
30.5
61
31.0
62
31.5
63
32.0
64
32.5
65
33.0
66
33.5
67
34.0
68
34.5
69
35.0
70
35.5
71
36.0
72
36.5
73
37.0
74
37.5
75
38.0
76
38.5
77
39.0
78
39.5
79
40.0
80
40.5
81
41.0
82
41.5
83
42.0
84
42.5
85
43.0
86
43.5
87
44.0
88
44.5
89
45.0
90
45.5
91
46.0
92
46.5
93
47.0
94
47.5
95
48.0
96
48.5
97
49.0
98
49.5
99
50.0
100
50.5
101
51.0
102
51.5
103
52.0
104
52.5
105
53.0
106
53.5
107
54.0
108
54.5
109
55.0
110
55.5
111
56.0
112
56.5
113
57.0
114
57.5
115
58.0
116
58.5
117
59.0

### Ejercicio 3

Utilizamos la funcion gradiente_descendiente para calcular el w optimo y b optimo de el conjunto de entrenamiento, este calcula los errores a medida que se van actualizando los valores de w y b

In [ ]:
def error_cuadratico_medio(imagenes, w, b):
    imagenes_normal = imagenes[0]
    imagenes_pneumonia = imagenes[1]

    error_total = 0
    for i in imagenes_normal:
        error = (f(i, w, b) - 0)**2
        error_total += error
    
    for i in imagenes_pneumonia:
        error = (f(i, w, b) - 1)**2
        error_total += error

    return error_total/len(imagenes_normal + imagenes_pneumonia)

In [ ]:
imagenes_normal = abrirImagenesEscaladas("chest_xray/train/NORMAL")
imagenes_pneumonia = abrirImagenesEscaladas("chest_xray/train/PNEUMONIA")
imagenes = [imagenes_normal, imagenes_pneumonia]

w_opt_train, b_opt_train, errores_train = gradiente_descendiente(imagenes, 0.001)
print(errores_train)

UnidentifiedImageError: cannot identify image file 'chest_xray/train/PNEUMONIA\\.DS_Store'

### Ejercicio 4

El parametro alpha es muy importante a la hora de encontrar el minimo. Este determina cuanto vamos a movernos en dirección al gradiente. Si este es muy chico, encontrar el minimo va a tardar muchos iteración y, dependiendo de el valor maximo, es posible que no se encuentre. Si se elige un valor muy grande, es posible que nos pasemos del valor minimo local.

In [ ]:
alpha = [0.0000001, 0.0001, 0.01, 1, 10]
for a in alpha:
    w_opt, b_opt = gradiente_descendiente(imagenes, a)
    print("Alpha: ", a, "- L = ", L(imagenes, w_opt, b_opt), " - Error cuadrático medio: ", error_cuadratico_medio(imagenes, w_opt, b_opt))

Iteración:  0 - Mínimo alcanzado hasta el momento:  60.67761335170323
w: [-3.06642722e-07 -4.30690102e-07 -4.30929655e-07 ...  1.15433243e-07
  1.55135632e-07  1.69193019e-07]
b: 0.4999963656900931
L nuevo: 53.07365823098891
L viejo: 60.67761335170323
diferencia L: 7.603955120714318
Iteración:  1 - Mínimo alcanzado hasta el momento:  53.07365823098891
w: [-1.81921134e-07 -4.30774881e-07 -4.34814820e-07 ...  4.02058690e-07
  5.08398319e-07  5.49468263e-07]
b: 0.4999940570316326
L nuevo: 50.42672157639585
L viejo: 53.07365823098891
diferencia L: 2.6469366545930626
Iteración:  2 - Mínimo alcanzado hasta el momento:  50.42672157639585
w: [ 3.04826898e-07 -5.81612696e-08 -6.84936028e-08 ...  8.09714086e-07
  1.00032642e-06  1.07714779e-06]
b: 0.4999930844972514
L nuevo: 49.771893872707835
L viejo: 50.42672157639585
diferencia L: 0.6548277036880137
Iteración:  3 - Mínimo alcanzado hasta el momento:  49.771893872707835
w: [9.30270381e-07 4.59395080e-07 4.41864898e-07 ... 1.25962754e-06
 1.540

KeyboardInterrupt: 

### Ejercicio 5

In [ ]:
escalado = [16, 32, 64, 128, 256, 512]

for e in escalado:
    imagenes_normal = abrirImagenesEscaladas("chest_xray/small_train/NORMAL", e)
    imagenes_pneumonia = abrirImagenesEscaladas("chest_xray/small_train/PNEUMONIA", e)
    imagenes = [imagenes_normal, imagenes_pneumonia]

    tiempo_inicial = time.time()
    w_opt, b_opt = gradiente_descendiente(imagenes, 0.001)
    tiempo_final = time.time()
    print("Tiempo de ejecución: ", tiempo_final-tiempo_inicial)
    print("Escala: ", e, "- L = ", L(imagenes, w_opt, b_opt), "- Error cuadrático medio: ", error_cuadratico_medio(imagenes, w_opt, b_opt))

Iteración:  0 - Mínimo alcanzado hasta el momento:  50.0
Iteración:  1 - Mínimo alcanzado hasta el momento:  47.37579165693172
Iteración:  2 - Mínimo alcanzado hasta el momento:  54.40794117059423
Iteración:  3 - Mínimo alcanzado hasta el momento:  96.60520972069014
Iteración:  4 - Mínimo alcanzado hasta el momento:  92.70278868357126
Iteración:  5 - Mínimo alcanzado hasta el momento:  69.75197786150365
Iteración:  6 - Mínimo alcanzado hasta el momento:  94.28938051439916
Iteración:  7 - Mínimo alcanzado hasta el momento:  79.86667296177323
Iteración:  8 - Mínimo alcanzado hasta el momento:  74.54716144111538
Iteración:  9 - Mínimo alcanzado hasta el momento:  87.4604925217846
Iteración:  10 - Mínimo alcanzado hasta el momento:  37.70919849237352
Iteración:  11 - Mínimo alcanzado hasta el momento:  37.5991346070351
Iteración:  12 - Mínimo alcanzado hasta el momento:  72.82041175355346
Iteración:  13 - Mínimo alcanzado hasta el momento:  87.60263641799033
Iteración:  14 - Mínimo alcanza

### Ejercicio 6